In [1]:
import sys

In [2]:
sys.path.append('../')

In [3]:
from model.model import FineTuner
from model.dataloader import DataModule

/home2/shivprasad.sagare/miniconda3/envs/xalign_role/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dm_args = {
    'train_path': '../data/xalign_unified_script/train.csv',
    'val_path': '../data/xalign_unified_script/val.csv',
    'test_path': '../data/xalign_unified_script/test.csv',
    'tokenizer_name_or_path': 'google/mt5-small',
    'max_source_length': 384,
    'max_target_length': 128,
    'train_batch_size': 4,
    'val_batch_size': 4,
    'test_batch_size': 4,
}

In [5]:
dm = DataModule(**dm_args)

/home2/shivprasad.sagare/miniconda3/envs/xalign_role/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


we added S|, P|, O| to the tokenizer


In [6]:
model_args = {
    'learning_rate': 3e-5,
    'model_name_or_path': 'google/mt5-small',
    'eval_beams': 4,
    'tgt_max_seq_len': 128,
    'tokenizer': dm.tokenizer,
}

In [ ]:
model = FineTuner(**model_args)

In [8]:
dm.setup(stage='fit')

In [9]:
for batch in dm.train_dataloader():
    batch1 = batch
    break

In [10]:
input_ids, attention_mask, labels = batch1['input_ids'], batch1['attention_mask'],  batch1['labels']

In [11]:
outputs = model(input_ids, attention_mask, labels)

In [12]:
outputs.loss

tensor(inf, grad_fn=<NllLossBackward0>)

In [13]:
logits = outputs.logits

In [24]:
logits[3][127]

tensor([-77.7112,  -4.4019, -77.8719,  ..., -77.3091,  -4.1475,  -4.0577],
       grad_fn=<SelectBackward0>)

In [16]:
tok = model.hparams.tokenizer

In [36]:
tok.decode([195000])

'ివి'

In [37]:
model

FineTuner(
  (model): MT5CopyGenerator(
    (shared): Embedding(250103, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(250103, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=384, bias=False)
                (k): Linear(in_features=512, out_features=384, bias=False)
                (v): Linear(in_features=512, out_features=384, bias=False)
                (o): Linear(in_features=384, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 6)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=512, out_features=1024, bias=False)
                (wi_1): Linear(in_features=